In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:33942")
client

In [ ]:
import dask.dataframe as dd

df = dd.read_csv("../../../data/train/viznet/all.csv", keep_default_na=False, dtype=str)

In [ ]:
import csv
import nltk
import numpy as np

df = df.persist()

i = 2

def ngrams(x):
    tokens = nltk.wordpunct_tokenize(x)
    return tokens

df[f"{i}_gram"] = df["data"].apply(ngrams, meta=(f"{i}_gram", np.array([""])))
ngram_df = df.explode(f"{i}_gram")
value2count = ngram_df[f"{i}_gram"].value_counts()
value2count.index.name = "data"
value2count.to_frame("count").to_csv(
    f"../../../data/train/viznet/ngram/token_count.csv",
    single_file=True,
    quoting=csv.QUOTE_ALL,
)
df = df.drop(f"{i}_gram", axis=1)